In [1]:
import sys, os
import numpy as np
from time import time as t
import cupy as cp

sys.path.append('../src/')
from main_structures import Grammar, Graph, Equation

In [2]:
grammars_path ='../data/grammars/rdf.g1_cnf (prob).txt'
grm = Grammar(grammars_path, grammar_type='Prob')

graph_path = '../data/graphs/'
graph_paths = os.listdir(graph_path)

ll_g1={'generations.txt': 2164, 'travel.txt': 2499, 'biomedical-mesure-primitive.txt': 15156,
    'atom-primitive.txt': 15454, 'foaf.txt': 4118, 'funding.txt': 17634, 'univ-bench.txt': 2540,
    'pizza.txt': 56195, 'skos.txt': 810, 'wine.txt': 66572, 'people_pets.txt': 9472, 'core.txt':204}
k = 1 #grm.ntc

{'S': 0, 'C1': 2, 'C2': 6} 3
{'t_r': 1, 't': 3, 'sco_r': 4, 'sco': 5} 4
{0: [[1, 2, 0.2], [4, 5, 0.25], [4, 6, 0.4], [1, 3, 0.15]], 2: [[0, 3, 1.0]], 6: [[0, 5, 1.0]]}


In [3]:
def equation_g1(eq, mtx):
    mtx = (0.2 * eq.matrices[1].dot(mtx).dot(eq.matrices[3]) + 0.15 * eq.matrices[1].dot(eq.matrices[3]) + 0.4 *
           eq.matrices[4].dot(mtx).dot(eq.matrices[5]) + 0.25 * eq.matrices[4].dot(eq.matrices[5])) - mtx
    return mtx

In [4]:
for gp in graph_paths:
    if gp[-4:]!='.txt': continue 
    with cp.cuda.Device(0):
        s = t()
        mempool = cp.get_default_memory_pool()
        mempool.free_all_blocks()
        print('Graph: ', gp)
        grh = Graph(graph_path + gp)
        grh.fill(grm)
        eq = Equation(grm, grh)
        
        def equation(mtx):
            return equation_g1(eq, mtx)
        print('Fill time', round((t() - s) * 1000, 2), 'ms')

        s = t()
        res = eq.newton_krylov(equation, info=True)
        s2 = t()
        numb = grm.nonterminals['S']
        nnz = res.count_nonzero()
    print(gp,' - ', nnz, '/', ll_g1[gp], ' - ', round((s2 - s) * 1000, 2), 'ms')
    print()

Graph:  generations.txt
Fill time 425.02 ms
0:  |F(x)| = 8.90657e-08; step 1
1:  |F(x)| = 3.83924e-15; step 1
generations.txt  -  2164 / 2164  -  617.16 ms

Graph:  pizza.txt
Fill time 20.06 ms
0:  |F(x)| = 2.59205e-05; step 1
1:  |F(x)| = 5.30675e-12; step 1
2:  |F(x)| = 1.14793e-14; step 1
3:  |F(x)| = 2.804e-15; step 1
pizza.txt  -  56195 / 56195  -  2019.12 ms

Graph:  foaf.txt
Fill time 13.59 ms
0:  |F(x)| = 1.02185e-06; step 1
1:  |F(x)| = 4.78396e-14; step 1
2:  |F(x)| = 9.35491e-16; step 1
foaf.txt  -  4118 / 4118  -  930.12 ms

Graph:  people_pets.txt
Fill time 12.22 ms
0:  |F(x)| = 1.36631e-06; step 1
1:  |F(x)| = 7.46019e-14; step 1
2:  |F(x)| = 6.38076e-16; step 1
people_pets.txt  -  9472 / 9472  -  1040.7 ms

Graph:  univ-bench.txt
Fill time 13.0 ms
0:  |F(x)| = 6.01639e-07; step 1
1:  |F(x)| = 4.06697e-14; step 1
2:  |F(x)| = 8.3728e-16; step 1
univ-bench.txt  -  2540 / 2540  -  918.91 ms

Graph:  atom-primitive.txt
Fill time 16.14 ms
0:  |F(x)| = 1.71397e-05; step 1
1:  